# Pergunta 1: Qual FII negociou o maior volume em um único dia?

In [1]:
import pandas as pd

# Altere aqui o caminho do arquivo "negociacao_fiis" para rodar o codigo
FILE_PATH = r'C:\Users\luish\OneDrive\Área de Trabalho\Python Projetos\case-multiestrategia\dados\01_cleaned_data\negociacao_fiis.xlsx'

def carregar_dados_negociacao():
    try:
        df_negociacao = pd.read_excel(FILE_PATH)
    except FileNotFoundError as e:
        raise FileNotFoundError(f"Arquivo não encontrado: {e}")

    df_negociacao['dt_pregao'] = pd.to_datetime(df_negociacao['dt_pregao'], dayfirst=True)
    return df_negociacao

# Função para encontrar o FII com maior volume negociado em um único dia
def encontrar_fii_maior_volume(df):
    max_row = df.loc[df['volume_negociado'].idxmax()]
    return max_row['cod_negociacao_papel'], max_row['dt_pregao'], max_row['volume_negociado']

df_negociacao = carregar_dados_negociacao()
fii_max, date_max, volume_max = encontrar_fii_maior_volume(df_negociacao)

print(f"FII com maior volume negociado: {fii_max}")
print(f"Data: {date_max.strftime('%d/%m/%Y')}")
print(f"Volume negociado: R$ {volume_max:,.2f}")


FII com maior volume negociado: BTLG11
Data: 15/03/2024
Volume negociado: R$ 219,392,617.08


# Pergunta 2: Em qual dia de 2024 tivemos o maior volume negociado por FIIs? Qual foi esse volume?

In [2]:
# Filtrar e encontrar o dia com maior volume total negociado em 2024
def encontrar_maior_volume(df):
    df_2024 = df[df['dt_pregao'].dt.year == 2024]  # Filtrar apenas registros de 2024 para se adequar a pergunta, caso a base tivesse mais anos
    daily_volume = df_2024.groupby('dt_pregao')['volume_negociado'].sum()
    max_daily_date = daily_volume.idxmax()
    max_daily_volume = daily_volume.max()
    return max_daily_date, max_daily_volume

max_daily_date, max_daily_volume = encontrar_maior_volume(df_negociacao)

print(f"Dia com maior volume total negociado: {max_daily_date.strftime('%d/%m/%Y')}")
print(f"Volume total negociado: R$ {max_daily_volume:,.2f}".replace(',', 'X').replace('.', ',').replace('X', '.'))


Dia com maior volume total negociado: 20/09/2024
Volume total negociado: R$ 973.852.736,23


# Pergunta 3: Qual o volume negociado pelo IFIX em cada mês de 2024?

In [3]:
IFIX_FILE_PATH = r'C:\Users\luish\OneDrive\Área de Trabalho\Python Projetos\case-multiestrategia\dados\01_cleaned_data\carteira_ifix.xlsx'

def carregar_dados_ifix():
    df_ifix = pd.read_excel(IFIX_FILE_PATH)
    df_ifix['dt_pregao'] = pd.to_datetime(df_ifix['dt_pregao'], dayfirst=True)
    return df_ifix

def calcular_volume_ifix_por_mes(df_negociacao, df_ifix):

    """
    O merge com 'dt_pregao' faz a verificação diária da composição do IFIX.
    Fiz desse modo pois um FII pode entrar ou sair do índice em qualquer dia do mês.
    Se fosse apenas um agrupamento mensal, teriamos o erro de considerar o volume de ativos em dias em que eles não faziam parte do índice.
    
    """

    df_negociacao = df_negociacao.copy()
    df_negociacao['dt_pregao'] = pd.to_datetime(df_negociacao['dt_pregao'])
    df_negociacao['mes_ano'] = df_negociacao['dt_pregao'].dt.strftime('%m/%Y')
    
    df_volumes = (
        df_negociacao
        .merge(
            df_ifix[['dt_pregao', 'ticker']],
            left_on=['dt_pregao', 'cod_negociacao_papel'],
            right_on=['dt_pregao', 'ticker'],
            how='inner'
        )
        .groupby('mes_ano')['volume_negociado']
        .sum()
        .to_dict()
    )
    
    return df_volumes
# Carrega dados e calcula volumes
df_ifix = carregar_dados_ifix()
monthly_ifix_volume = calcular_volume_ifix_por_mes(df_negociacao, df_ifix)

# Imprime resultados
print("\nVolume negociado pelo IFIX em cada mês de 2024:")
for month, volume in monthly_ifix_volume.items():
    formatted_volume = f"{volume:,.2f}".replace(",", "X").replace(".", ",").replace("X", ".")
    print(f"{month}: R$ {formatted_volume}")


Volume negociado pelo IFIX em cada mês de 2024:
01/2024: R$ 5.778.284.556,55
02/2024: R$ 5.521.304.181,15
03/2024: R$ 5.600.778.185,72
04/2024: R$ 5.926.781.382,37
05/2024: R$ 5.927.519.243,75
06/2024: R$ 5.840.800.301,04
07/2024: R$ 6.157.876.737,25
08/2024: R$ 5.579.931.091,52
09/2024: R$ 6.526.389.776,50
10/2024: R$ 7.133.783.943,50
11/2024: R$ 5.755.659.888,66
12/2024: R$ 7.067.464.772,93


# Pergunta 4: No dia de maior volume negociado, qual porcentagem desse volume foi proveniente dos fundos que compõem o IFIX?

In [4]:
# Calcular a porcentagem do volume dos FIIs do IFIX
def calcular_percentual_ifix(df_negociacao, df_ifix, max_daily_date, max_daily_volume):
    df_max_day = df_negociacao[df_negociacao['dt_pregao'] == max_daily_date]
    df_ifix_day = df_ifix[df_ifix['dt_pregao'] == max_daily_date]
    ifix_fiis = df_ifix_day['ticker'].unique()
    ifix_volume = df_max_day[df_max_day['cod_negociacao_papel'].isin(ifix_fiis)]['volume_negociado'].sum()
    return (ifix_volume / max_daily_volume) * 100

df_ifix = carregar_dados_ifix()
ifix_percentage = calcular_percentual_ifix(df_negociacao, df_ifix, max_daily_date, max_daily_volume)

print(f"\nNo dia {max_daily_date.strftime('%d/%m/%Y')}, {ifix_percentage:.2f}% do volume negociado veio dos FIIs que compõem o IFIX.")



No dia 20/09/2024, 89.84% do volume negociado veio dos FIIs que compõem o IFIX.
